### WikiEngine

In [1]:
from wiki.dump import WikiDump

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = WikiDump(path_xml, path_index)

### Exploring the wiki dump

In [2]:
dump.load_page('Portal:Physics/Topics')
dump.links[:5]

WikiDump: Loading index...
WikiDump: Loaded.


['Classical physics', 'mechanics', 'optics', 'electricity', 'magnetism']

In [7]:
dump.load_page('Danielle Bassett')
dump.links[:3]

['University of Pennsylvania',
 'Pennsylvania State University',
 'University of Cambridge']

In [9]:
dump.load_page('Matter', filter_top=True).strip_code()[:200]

'In classical physics and general chemistry, matter is any substance that has mass and takes up space by having volume. All everyday objects that can be touched are ultimately composed of atoms, which '

In [10]:
dump.load_page('Index of physics articles (P)')
dump.links[50:54]

['Pandemonium effect', 'Panemone windmill', 'Panofsky Prize', 'Pantur Silaban']

In [11]:
# testing XML cache
%time page = dump.load_page('AccessibleComputing')
%time page = dump.load_page('Anarchism')
%time page = dump.load_page('Angola')

CPU times: user 92.2 ms, sys: 1.79 ms, total: 94 ms
Wall time: 93 ms
CPU times: user 71.1 ms, sys: 211 µs, total: 71.3 ms
Wall time: 71.2 ms
CPU times: user 228 ms, sys: 1.3 ms, total: 230 ms
Wall time: 230 ms


### Wikipedia hypernet traversal

In [ ]:
import networkx as nx

class WikiCrawler():
    @staticmethod
    def bfs(graph, dump, queue, depth_goal=1):
        depth = 0
        depth_num_items = len(queue)
        depth_inc_pending = False
        while queue:
            page = queue.pop(0)
            depth_num_items -= 1
            if depth_num_items == 0:
                depth += 1
                print('Depth: ' + str(depth))
                depth_inc_pending = True
            if dump.load_page(page, filter_top=True):
                for link in dump.links:
                    link = str(link).split('#')[0].capitalize()
                    if (page, link) not in graph.edges:
                        graph.add_edge(link, page, 
                                       weight=1)
                        queue.append(link)
            if depth_inc_pending:
                depth_num_items = len(queue)
                depth_inc_pending = False
            if depth == depth_goal:
                break

In [ ]:
graph = nx.DiGraph()
queue = ['Matter']
WikiCrawler.bfs(graph, dump, queue, 3)

In [ ]:
len(graph.nodes)

### Port to gephi

In [ ]:
nx.write_gexf(graph, path_base + 'graphs/matter_d3.gexf')

### Edge weights